In [167]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
from sklearn.metrics import mean_squared_error

import itertools
import pickle
import random

import warnings
warnings.simplefilter(action='ignore', category=Warning)

## CREDIT LIMIT MODEL

### Import dataset

In [168]:
df = pd.read_csv('./prediction_model/data_source/data_month_full_T52024_test.csv')
df['based_month'] = pd.to_datetime(df['based_month'])

### Preprocessing data

In [169]:
# Detect các case không phát sinh rev tại bất kì tháng nào
group_ret = df.groupby('retailer_id')['gmv'].sum()
group_ret[group_ret == 0]

# Loại bỏ các case trên
df = pd.merge(df, group_ret, left_on = 'retailer_id', right_on='retailer_id')
df = df[df['gmv_y'] != 0]
# df = df.set_index(['based_month'])
df = df.drop(columns=['gmv_y']).rename(columns = {'gmv_x' : 'gmv'})

In [170]:
group_ret

retailer_id
47056     1.388509e+11
114816    9.563299e+10
140533    8.372336e+10
Name: gmv, dtype: float64

In [171]:
# list_ret = list(df['retailer_id'].unique())
# random.seed(42)
# random_values = random.sample(list_ret,1)
# list_ret_0_10 = random_values[0:1]

# df_test = df[df['retailer_id'] == 4119]

### Checking stationary

In [172]:
# Create function to find optimal differencing with lowest p-value with ADFuller test stationary
def checking_stationary(df, diff):
    best_diff = None
    min_pvalue = float('inf')
    for i in range(diff, diff + 3):
        diff_df = df['gmv'] - df['gmv'].shift(i)
        res = adfuller(diff_df.dropna())
        p_value = res[1]
        if p_value < min_pvalue:
            min_pvalue = p_value
            best_diff = i
    return best_diff, min_pvalue

### Storing best params

In [173]:
# Function to train SARIMAX model and find the best parameters using mean squared error
def find_best_params(customer_data, periods, param_grid):
    best_params = {}

    for period in periods:
        # Define differencing value
        diff = checking_stationary(customer_data, period)[0]
        # Convert data to be stationary
        shift_data = customer_data['gmv'] - customer_data['gmv'].shift(diff)
        shift_data = shift_data.dropna()

        best_mse = np.inf
        best_param = None

        train_size = len(shift_data) - period
        train, test = shift_data[:train_size], shift_data[train_size:]

        for param in param_grid:
            order, seasonal_order = param
            try:
                model = SARIMAX(train, order=order, seasonal_order=seasonal_order, enforce_stationarity=False)
                results = model.fit(disp=False)
                forecast = results.forecast(steps=period)
                mse = mean_squared_error(test, forecast)
                if mse < best_mse:
                    best_mse = mse
                    best_param = param
            except:
                continue
        best_params[period] = best_param
    return best_params

# Define the parameter grid for SARIMAX
p = range(1, 4)  
d = range(0, 2)  
q = range(1, 4)  
P = range(1, 4)  
D = range(0, 2)  
Q = range(1, 4)  
s = [6, 9, 12]

param_grid = list(itertools.product(itertools.product(p, d, q), itertools.product(P, D, Q, s)))

# Define periods for prediction
periods = [1, 3, 6, 12]

# Dictionary to store best parameters for each customer
customer_best_params = {}

# Find and save best parameters for each customer
for customer_id, customer_data in df.groupby('retailer_id'):
    best_params = find_best_params(customer_data, periods, param_grid)
    customer_best_params[customer_id] = best_params
    print(f'Finish finding the best param for {customer_id}')

# Save the best parameters to a pickle file
with open('D:/Project/KLending/New_Lending/ARIMA_model/best_param/best_params.pkl', 'wb') as f:
    pickle.dump(customer_best_params, f)

# Function to load best parameters from pickle file
def load_best_params(pickle_file):
    with open(pickle_file, 'rb') as f:
        return pickle.load(f)

# Load the best parameters
loaded_params = load_best_params('D:/Project/KLending/New_Lending/ARIMA_model/best_param/best_params.pkl')

Finish finding the best param for 47056
Finish finding the best param for 114816
Finish finding the best param for 140533


In [175]:
customer_best_params

{47056: {1: ((2, 0, 3), (2, 0, 2, 6)),
  3: ((1, 0, 1), (1, 0, 1, 9)),
  6: ((3, 0, 1), (1, 0, 1, 6)),
  12: None},
 114816: {1: ((1, 0, 2), (2, 0, 2, 12)),
  3: ((2, 0, 1), (2, 0, 2, 6)),
  6: ((1, 1, 1), (1, 0, 1, 6)),
  12: ((1, 0, 1), (1, 1, 1, 6))},
 140533: {1: ((1, 1, 1), (3, 0, 3, 6)),
  3: ((1, 0, 1), (1, 1, 3, 6)),
  6: ((1, 0, 3), (1, 0, 1, 6)),
  12: ((1, 0, 2), (1, 0, 1, 6))}}

In [174]:
load_best_params('D:/Project/KLending/New_Lending/ARIMA_model/best_param/best_params.pkl')

{47056: {1: ((2, 0, 3), (2, 0, 2, 6)),
  3: ((1, 0, 1), (1, 0, 1, 9)),
  6: ((3, 0, 1), (1, 0, 1, 6)),
  12: None},
 114816: {1: ((1, 0, 2), (2, 0, 2, 12)),
  3: ((2, 0, 1), (2, 0, 2, 6)),
  6: ((1, 1, 1), (1, 0, 1, 6)),
  12: ((1, 0, 1), (1, 1, 1, 6))},
 140533: {1: ((1, 1, 1), (3, 0, 3, 6)),
  3: ((1, 0, 1), (1, 1, 3, 6)),
  6: ((1, 0, 3), (1, 0, 1, 6)),
  12: ((1, 0, 2), (1, 0, 1, 6))}}

### SARIMAX model function

In [146]:
def sarimax_model(retailer_id, best_params, data):
    customer_data = data[data['retailer_id'] == retailer_id]
    sum_predictions = {} # Save predicted output of 1, 3, 6, 12 months
    for period, param in best_params.items():
        # Making data to be stationary
        diff = checking_stationary(customer_data, period)[0]
        shift_df = customer_data['gmv'] - customer_data['gmv'].shift(diff)
        shift_df = shift_df.dropna()

        try:
            # define order and seasonal order
            order, seasonal_order = param
            #Run model
            model = SARIMAX(shift_df, order=order, seasonal_order=seasonal_order, enforce_stationarity=False)
            results = model.fit(disp=False)
            prediction = results.forecast(steps=period) # thực hiện predict revenue in next X month
            forecast_origin = prediction + pd.Series(customer_data['gmv'][-diff:][:period].values, index = prediction.index)  # convert value back to original value
            sum_predictions[f'forecast_gmv_{period}_month'] = forecast_origin.apply(lambda x: 0 if x < 0 else x).sum() # Nếu kết quả predict ra revenue âm thì replace = 0
        except TypeError as error:
            print(f"The revenue for retailer_id: {retailer_id} cannot be predicted for the next {period} months.")
            continue

    #Create Dataframe contains predicted output of 1, 3, 6, 12 months
    results_df = pd.DataFrame(sum_predictions.items()).T 
    results_df.columns = results_df.iloc[0]
    results_df = results_df[1:]
    results_df['retailer_id'] = retailer_id

    return results_df

### Running all merchants

In [147]:
forecast_rev = pd.DataFrame()
for retailer_id, best_param in loaded_params.items():    
    try:
        ret_df = sarimax_model(retailer_id, best_param, df)
        forecast_rev = pd.concat([forecast_rev, ret_df], ignore_index = False)
        print("Finish retailer_id {}".format(retailer_id))
    except ValueError as error:
        print("ValueError occurred for retailer_id {}: {}".format(retailer_id, error))
        continue

The revenue for retailer_id: 47056 cannot be predicted for the next 12 months.
Finish retailer_id 47056
Finish retailer_id 114816
Finish retailer_id 140533


In [148]:
forecast_rev

,forecast_gmv_1_month,forecast_gmv_3_month,forecast_gmv_6_month,retailer_id,forecast_gmv_12_month
1,0.0,12555332055.872406,90395636729.119949,47056,NaN
1,4309143778.40907,19475310915.66555,24199180100.811882,114816,51152949082.359856
1,2033195558.707653,9125768781.98715,18340315152.239944,140533,32519846636.412407


## RELIABILITY SCORING MODEL

### Import dataset

In [162]:
bus_data = pd.read_csv('C:/Users/hung.vv1/Desktop/streamlit_new_lending/hung_test/data_source/backtest/T5_2024/reliability_business_metrics.csv')
usage_data = pd.read_csv('C:/Users/hung.vv1/Desktop/streamlit_new_lending/hung_test/data_source/backtest/T5_2024/reliability_usage_metrics.csv')
gr_rev_inv_3_6_12 = pd.read_csv('C:/Users/hung.vv1/Desktop/streamlit_new_lending/hung_test/data_source/backtest/T5_2024/growth_rate_rev_inv_3_6_12.csv')
gr_pur_3_6_12 = pd.read_csv('C:/Users/hung.vv1/Desktop/streamlit_new_lending/hung_test/data_source/backtest/T5_2024/growth_rate_pur_3_6_12.csv')

### Preprocessing data

In [163]:
# Tạo dataset chỉ số kinh doanh
# Combine các bảng dataset chỉ số kinh doanh
df_list = [forecast_rev, gr_rev_inv_3_6_12, gr_pur_3_6_12]

df_combine = df_list[0]
for df_ in df_list[1:]:
    df_combine = df_combine.merge(df_, how='left', on='retailer_id')

fin_data = df_combine


In [164]:
# Label gói hợp đồng phần mềm và fillna data hành vi sử dụng KV
usage_data['goi_HDPM_cuoi_cung'] = usage_data['goi_HDPM_cuoi_cung'].replace(['Other','Hỗ trợ','Chuyên nghiệp', 'Cao cấp'],
                                                                            [0,0,1,2])
usage_data = usage_data.fillna(0)

In [165]:
# Combine các bảng dataset 
df_list_2 = [forecast_rev[['retailer_id']], bus_data, usage_data, fin_data]

all_data = df_list_2[0]
for df_ in df_list_2[1:]:
    all_data = all_data.merge(df_, how='left', on='retailer_id')

all_data = all_data.fillna(0)


### Detect & remove outliers

In [166]:
# Replace all values > quantile 0.8 equal to quantile 0.75
final_data = all_data.copy()

def iqr(df):
    for col in df.columns[df.columns.isin(['cnt_product', 'avg_new_product',
       'avg_saled_product_per_day', 'avg_comeback_customer',
       'avg_rate_comeback_customer', 'avg_new_cus_per_month',
       'avg_customer_per_month', 'no_suppliers','thoihan','gr_rev_l3m', 'gr_rev_l6m', 'gr_rev_l12m',
       'gr_inv_l3m', 'gr_inv_l6m', 'gr_inv_l12m','gr_pur_l3m', 'gr_pur_l6m', 'gr_pur_l12m'])]:
        if df[col].dtype != object:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            S = 1.5 * IQR
            LB = Q1 - S
            UB = Q3 + S
            df.loc[df[col] > UB,col] = UB
            # df.loc[df[col] < LB,col] = LB
        else:
            break
    return df
detect_data_1 = iqr(final_data)

In [167]:
# Replace các numerical features có quá nhiều giá trị 0 về thành dạng Yes(1) or No(0)
def categ_outlier(df):
    for col in df.columns[df.columns.isin(['total_active_employee','cashflow_signal','pricebook_signal','stocktake_signal','number_of_ticket'])]:
        df.loc[df[col] > 0, col] = 1
    return df

detect_data_2 = categ_outlier(detect_data_1)

### Feature scaling

In [170]:
#Scaling down the data 
# demonstrate data standardization with sklearn

fs_final_data = detect_data_2

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
scaler_col = fs_final_data.columns[(fs_final_data.columns != 'retailer_id')]
fs_final_data[scaler_col] = min_max_scaler.fit_transform(fs_final_data[scaler_col])

### Weight of features

In [171]:
# Chỉ số hoạt động - 10%
  # Khách hàng - 40%
avg_customer_per_month_rate = 0.4 * 0.4
avg_comeback_customer_rate = 0.4 * 0.4
avg_rate_comeback_customer_rate = 0.4 * 0.1
avg_new_cus_per_month_rate = 0.4 * 0.1
  # Sản phẩm - 35%
avg_saled_product_per_day_rate = 0.35 * 0.4
cnt_product_rate = 0.35 * 0.4
avg_new_product_rate = 0.35 * 0.2
  # NCC - 15%
no_suppliers_rate = 0.15
  # Employee - 10%
total_active_employee_rate = 0.1

#-------------------------------------------------------------------

# Hành vi sử dụng - 40%
  # Số lượng module - 30%
invoice_module_rate = 0.3 * 0.4
product_module_rate = 0.3 * 0.2
stocktake_module_rate = 0.3 * 0.2
customer_module_rate = 0.3 * 0.1
addon_module_rate = 0.3 * 0.05
KVemployee_module_rate = 0.5 * 0.3 * 0.05
  # Tần suất sử dụng KV - 45%
invoice_signal_rate = 0.45 * 0.5
product_signal_rate = 0.45 * 0.2
purchase_order_signal_rate = 0.45 * 0.1
cashflow_signal_rate = 0.45 * 0.1
pricebook_signal_rate = 0.45 * 0.05
stocktake_signal_rate = 0.45 * 0.05
  # Thời gian sử dụng KV - 15%
time_usage_rate = 0.15 * 0.1
lifespan_usage_rate = 0.15 * 0.1
loadpage_signal_rate = 0.15 * 0.4
inapp_signal_rate = 0.15 * 0.4
  # Hợp đồng & Tái kí - 5%
renewal_times_rate = 0.05 * 0.5
goi_HDPM_cuoi_cung_rate = 0.05 * 0.25
thoihan_rate = 0.05 * 0.25
  # Tương tác giữa KH và KV - 5%
ticket_rate = 0.05

#-------------------------------------------------------------------

# Chỉ số kinh doanh - 50%
  # Biến động doanh thu - 70%
pct_rev_l3m_rate = 0.7 * 0.5
pct_rev_l6m_rate = 0.7 * 0.4
pct_rev_l12m_rate = 0.7 * 0.1
  # Biến động invoice - 15%
pct_inv_l3m_rate = 0.15 * 0.5
pct_inv_l6m_rate = 0.15 * 0.4
pct_inv_l12m_rate = 0.15 * 0.1
  # Biến động purchase - 15%
pct_pur_l3m_rate = 0.15 * 0.5
pct_pur_l6m_rate = 0.15 * 0.4
pct_pur_l12m_rate = 0.15 * 0.1

### Calculating reliability score

In [172]:
fs_final_data['business_score'] = fs_final_data['cnt_product'] * cnt_product_rate + \
                                fs_final_data['avg_new_product'] * avg_new_product_rate + \
                                fs_final_data['avg_saled_product_per_day'] * avg_saled_product_per_day_rate + \
                                fs_final_data['avg_comeback_customer'] * avg_comeback_customer_rate + \
                                fs_final_data['avg_rate_comeback_customer'] * avg_rate_comeback_customer_rate + \
                                fs_final_data['avg_new_cus_per_month'] * avg_new_cus_per_month_rate + \
                                fs_final_data['avg_customer_per_month'] * avg_customer_per_month_rate + \
                                fs_final_data['no_suppliers'] * no_suppliers_rate + \
                                fs_final_data['total_active_employee'] * total_active_employee_rate 

In [173]:
fs_final_data['usage_score'] = fs_final_data['invoice_module'] * invoice_module_rate + \
                                fs_final_data['product_module'] * product_module_rate + \
                                fs_final_data['stocktake_module'] * stocktake_module_rate + \
                                fs_final_data['customer_module'] * customer_module_rate + \
                                fs_final_data['addon_module'] * addon_module_rate + \
                                fs_final_data['is_KVemployee'] * KVemployee_module_rate + \
                                fs_final_data['invoice_signal'] * invoice_signal_rate + \
                                fs_final_data['purchase_order_signal'] * purchase_order_signal_rate + \
                                fs_final_data['product_signal'] * product_signal_rate + \
                                fs_final_data['cashflow_signal'] * cashflow_signal_rate + \
                                fs_final_data['pricebook_signal'] * pricebook_signal_rate + \
                                fs_final_data['stocktake_signal'] * stocktake_signal_rate + \
                                fs_final_data['time_usage'] * time_usage_rate + \
                                fs_final_data['lifespan_usage'] * lifespan_usage_rate + \
                                fs_final_data['loadpage_signal'] * loadpage_signal_rate + \
                                fs_final_data['inapp_signal'] * inapp_signal_rate + \
                                fs_final_data['renewal_times'] * renewal_times_rate + \
                                fs_final_data['goi_HDPM_cuoi_cung'] * goi_HDPM_cuoi_cung_rate + \
                                fs_final_data['thoihan'] * thoihan_rate + \
                                fs_final_data['number_of_ticket'] * ticket_rate

In [174]:
fs_final_data['finance_score'] = fs_final_data['gr_rev_l3m'] * pct_rev_l3m_rate + \
                                fs_final_data['gr_rev_l6m'] * pct_rev_l6m_rate + \
                                fs_final_data['gr_rev_l12m'] * pct_rev_l12m_rate + \
                                fs_final_data['gr_inv_l3m'] * pct_inv_l3m_rate + \
                                fs_final_data['gr_inv_l6m'] * pct_inv_l6m_rate + \
                                fs_final_data['gr_inv_l12m'] * pct_inv_l12m_rate + \
                                fs_final_data['gr_pur_l3m'] * pct_pur_l3m_rate + \
                                fs_final_data['gr_pur_l6m'] * pct_pur_l6m_rate + \
                                fs_final_data['gr_pur_l12m'] * pct_pur_l12m_rate

In [ ]:
# labeling 1-5 each features
fs_final_data['business_quantile'] = pd.qcut(fs_final_data['business_score'], q=5, labels=[1, 2, 3, 4, 5]).astype(int)
fs_final_data['usage_quantile'] = pd.qcut(fs_final_data['usage_score'], q=5, labels=[1, 2, 3, 4, 5]).astype(int)
fs_final_data['finance_quantile'] = pd.qcut(fs_final_data['finance_score'], q=5, labels=[1, 2, 3, 4, 5]).astype(int)

# Calculating reliability score
fs_final_data['reliability_score'] = fs_final_data['business_quantile'] * 0.1 + fs_final_data['usage_quantile'] * 0.4 + fs_final_data['finance_quantile'] * 0.5

# Convert score to k-star rating
condition = [
    (fs_final_data['reliability_score'] <= 1) | (fs_final_data['gr_rev_l3m'] <= -0.8) | (fs_final_data['gr_rev_l6m'] <= -0.8),
    (fs_final_data['reliability_score'] > 1) & (fs_final_data['reliability_score']  <= 2),
    (fs_final_data['reliability_score'] > 2) & (fs_final_data['reliability_score'] <= 3),
    (fs_final_data['reliability_score'] > 3) & (fs_final_data['reliability_score'] <= 4),
    (fs_final_data['reliability_score'] > 4)
]

values = [1, 2, 3, 4, 5]
fs_final_data['k-star'] = np.select(condition, values)

## FINAL LENDING OUTPUT

In [ ]:
final_output = pd.concat(fs_final_data[['retailer_id', 'reliability_score','k-star']], forecast_rev, how = 'left', on = 'retailer_id') 